In [ ]:
import geopandas as gpd

gdf = gpd.read_file("C:/Users/nm_ma/Desktop/Nasa-App-Challenge/NASA-Space-App-Challenge-2025/Machine Learning/MercuryData/MERCURY_nomenclature_center_pts.shp")

gdf.to_file("mercury_features.json", driver="GeoJSON")

print(" Converted to mercury_features.json")


✅ Converted to mercury_features.json


In [ ]:
import json

with open("mercury_features.json", "r", encoding="utf-8") as f:
    data = json.load(f)

simplified = []

for feature in data["features"]:
    props = feature["properties"]
    coords = feature["geometry"]["coordinates"]

    simplified.append({
        "name": props.get("name"),
        "type": props.get("type"),
        "diameter": props.get("diameter"),
        "coordinates": coords,
        "link": props.get("link")
    })

with open("mercury_simple.json", "w", encoding="utf-8") as f:
    json.dump(simplified, f, indent=2)

print(" Saved as mercury_simple.json")


✅ Saved as mercury_simple.json


In [ ]:
import json
import re
from rapidfuzz import process

with open("mercury_simple.json", "r", encoding="utf-8") as f:
    craters = json.load(f)

# Extract crater names
crater_names = [c["name"] for c in craters]

def parse_coordinates(text):
    """
    Try to extract latitude and longitude from user input
    """
    # Regex to capture numbers (like 10N 20E, -3.5 45.2, lat 12 lon -30)
    matches = re.findall(r"(-?\d+\.?\d*)", text)
    if len(matches) >= 2:
        lat = float(matches[0])
        lon = float(matches[1])
        return lat, lon
    return None

def find_nearest_crater(lat, lon):
    """
    Find crater nearest to given lat/lon
    """
    def distance(c):
        c_lat, c_lon = c["coordinates"]
        return (c_lat - lat) ** 2 + (c_lon - lon) ** 2

    crater = min(craters, key=distance)
    return crater

def find_crater(user_input):
    user_input = user_input.lower().strip()

    # 1. Handle "biggest" or "largest" queries
    if "biggest" in user_input or "largest" in user_input:
        crater = max(craters, key=lambda c: c.get("diameter", 0))
        return {
            "name": crater["name"],
            "coordinates": crater["coordinates"],
            "diameter": crater["diameter"],
            "link": crater["link"],
            "reason": "Largest crater found"
        }

    # 2. Handle coordinates
    coords = parse_coordinates(user_input)
    if coords:
        lat, lon = coords
        crater = find_nearest_crater(lat, lon)
        return {
            "name": crater["name"],
            "coordinates": crater["coordinates"],
            "diameter": crater["diameter"],
            "link": crater["link"],
            "reason": f"Closest crater to ({lat}, {lon})"
        }

    # 3. Fuzzy match crater names
    best_match, score, idx = process.extractOne(user_input, crater_names)
    if score > 60:
        crater = craters[idx]
        return {
            "name": crater["name"],
            "coordinates": crater["coordinates"],
            "diameter": crater["diameter"],
            "link": crater["link"],
            "reason": f"Name matched with score {score}"
        }

    return None


# --- Example queries ---
print(find_crater("freddie mercury crater"))    # Fuzzy name
print(find_crater("10N 20E"))                   # Coordinates
print(find_crater("lat -3.5 lon 45.2"))         # Coordinates
print(find_crater("biggest crater"))            # NLP query


{'name': 'Waters', 'coordinates': [254.5466, -8.9583], 'diameter': 15.0, 'link': 'http://planetarynames.wr.usgs.gov/Feature/15086', 'reason': 'Name matched with score 72.0'}
{'name': 'Kyōsai', 'coordinates': [4.8556, 25.1532], 'diameter': 39.0, 'link': 'http://planetarynames.wr.usgs.gov/Feature/15084', 'reason': 'Closest crater to (10.0, 20.0)'}
{'name': 'Sousa', 'coordinates': [0.634, 46.6927], 'diameter': 138.0, 'link': 'http://planetarynames.wr.usgs.gov/Feature/14973', 'reason': 'Closest crater to (-3.5, 45.2)'}
{'name': 'Borealis Planitia', 'coordinates': [32.6, 67.3], 'diameter': 3450.0, 'link': 'http://planetarynames.wr.usgs.gov/Feature/823', 'reason': 'Largest crater found'}


In [8]:
import json
import re
from rapidfuzz import process

# Load crater data
with open("mercury_simple.json", "r", encoding="utf-8") as f:
    craters = json.load(f)

# Extract crater names
crater_names = [c["name"] for c in craters]

# --- Helper functions ---
def parse_coordinates(text):
    """Extract lat/lon from text"""
    matches = re.findall(r"(-?\d+\.?\d*)", text)
    if len(matches) >= 2:
        return float(matches[0]), float(matches[1])
    return None

def find_nearest_craters(lat, lon, top_n=3):
    """Return top N nearest craters to given lat/lon"""
    def distance(c):
        c_lat, c_lon = c["coordinates"]
        return (c_lat - lat)**2 + (c_lon - lon)**2

    sorted_craters = sorted(craters, key=distance)
    return sorted_craters[:top_n]

# --- Main search function ---
def find_crater(user_input, top_n=1):
    user_input = user_input.lower().strip()

    # 1. Simple NLP queries: biggest/largest/smallest
    if "biggest" in user_input or "largest" in user_input:
        crater = max(craters, key=lambda c: c.get("diameter",0))
        return [{"name": crater["name"], "coordinates": crater["coordinates"], 
                 "diameter": crater["diameter"], "link": crater["link"], 
                 "reason": "Largest crater found"}]

    if "smallest" in user_input:
        crater = min(craters, key=lambda c: c.get("diameter",0))
        return [{"name": crater["name"], "coordinates": crater["coordinates"], 
                 "diameter": crater["diameter"], "link": crater["link"], 
                 "reason": "Smallest crater found"}]

    # 2. Coordinate search
    coords = parse_coordinates(user_input)
    if coords:
        lat, lon = coords
        nearest = find_nearest_craters(lat, lon, top_n=top_n)
        return [{"name": c["name"], "coordinates": c["coordinates"], 
                 "diameter": c["diameter"], "link": c["link"], 
                 "reason": f"Closest crater to ({lat},{lon})"} for c in nearest]

    # 3. Name fuzzy search
    best_match, score, idx = process.extractOne(user_input, crater_names)
    if score > 60:
        crater = craters[idx]
        return [{"name": crater["name"], "coordinates": crater["coordinates"], 
                 "diameter": crater["diameter"], "link": crater["link"], 
                 "reason": f"Name matched with score {score}"}]

    # No match
    return []

# --- Example queries ---
queries = [
    "freddie mercury crater",       # fuzzy name
    "10 -34",                       # coordinates
    "biggest crater",               # largest
    "smallest crater",              # smallest
    "crater near 60 -34"            # coordinates
]

for q in queries:
    print(f"\nQuery: {q}")
    results = find_crater(q, top_n=3)
    for r in results:
        print(r)



Query: freddie mercury crater
{'name': 'Waters', 'coordinates': [254.5466, -8.9583], 'diameter': 15.0, 'link': 'http://planetarynames.wr.usgs.gov/Feature/15086', 'reason': 'Name matched with score 72.0'}

Query: 10 -34
{'name': 'Debussy', 'coordinates': [12.5377, -33.9513], 'diameter': 81.0, 'link': 'http://planetarynames.wr.usgs.gov/Feature/14647', 'reason': 'Closest crater to (10.0,-34.0)'}
{'name': 'Turms Planitia', 'coordinates': [9.19, -31.05], 'diameter': 622.0, 'link': 'http://planetarynames.wr.usgs.gov/Feature/15554', 'reason': 'Closest crater to (10.0,-34.0)'}
{'name': 'Hesperis', 'coordinates': [5.0001, -45.0], 'diameter': 0.0, 'link': 'http://planetarynames.wr.usgs.gov/Feature/2489', 'reason': 'Closest crater to (10.0,-34.0)'}

Query: biggest crater
{'name': 'Borealis Planitia', 'coordinates': [32.6, 67.3], 'diameter': 3450.0, 'link': 'http://planetarynames.wr.usgs.gov/Feature/823', 'reason': 'Largest crater found'}

Query: smallest crater
{'name': 'Caduceata', 'coordinates